In [1]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import time
###
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend
from tensorflow.keras.applications import VGG19
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
###
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
###
from utils import process_mnist

# I. Benchmark with MNIST dataset

In [2]:
X, y = fetch_openml('mnist_784', version = 1, return_X_y = True, as_frame = False)
X = process_mnist.resize_mnist(X, 32, 28)
###
X = np.stack((X,) * 3, axis = -1)
y = to_categorical(y, num_classes = 10)

In [3]:
assert X.shape == (70000, 32, 32, 3), "X should have a dimension of (70000, 32, 32, 3)"
assert y.shape == (70000,10), "y should have a dimension of (70000,10)"

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = None, test_size = 10000)

In [6]:
backend.clear_session()
try:
    os.mkdir('classifiers')
except:
    pass
full_name = 'Model A'
###
convolution_base = VGG19(weights = 'imagenet', include_top = False, 
                         input_shape = (32,32,3))
convolution_base.trainable = False
###
model = Sequential()
model.add(convolution_base)
x = model.output
###
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = GlobalAveragePooling2D()(x)
###
x = Dense(4096, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
x = Dropout(0.5)(x)
x = Dense(128, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
out = Dense(10, activation = 'softmax')(x)
finalModel = Model(inputs = model.input, outputs = out)
###
finalModel.summary()
###
opt = Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
finalModel.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy', 'Precision', 'Recall'])
###
filepath = "classifiers/%s-{epoch:02d}-{val_accuracy:.4f}-MNIST.hdf5"%full_name
checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy', save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19_input (InputLayer)     [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 1, 128)         589952    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 128)         147584    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 128)         147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 1, 128)         147584    
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0     

start = time.time()
history = finalModel.fit(X_train, y_train, epochs = 100, verbose = 1, callbacks = callback_list,
                         shuffle = True, validation_data = (X_test, y_test))
end = time.time()
runtime_mnist = end - start
###
print(f'Total runtime: {round(runtime_mnist, 2)}')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model A's Accuracy")
plt.ylabel('Accuracy (%)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()
###
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model A's Accuracy")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.grid(True)
plt.show()

## II. Benckmark with Fashion-MNIST dataset:

In [8]:
X_train, y_train = process_mnist.load_mnist('data/fashion', kind = 'train')
X_test, y_test = process_mnist.load_mnist('data/fashion', kind = 't10k')
###
X_train = process_mnist.resize_mnist(X_train, 32, 28)
X_train = np.stack((X_train,) * 3, axis = -1)
X_test = process_mnist.resize_mnist(X_test, 32, 28)
X_test = np.stack((X_test,) * 3, axis = -1)
###
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [9]:
assert X_train.shape == (60000,32,32,3), "X_train should have a dimension of (60000,32,32,3)!"
assert X_test.shape == (10000,32,32,3), "X_test should have a dimension of (10000,32,32,3)!"
assert y_train.shape == (60000,10) "y_train should have a dimension of (60000,10)"
assert y_test.shape == (10000,10) "y_test should have a dimension of (10000,10)"

SyntaxError: invalid syntax (<ipython-input-9-d054f5a51ecb>, line 3)

In [ ]:
start = time.time()
history = finalModel.fit(X_train, y_train, epochs = 100, verbose = 1, callbacks = callback_list,
                         shuffle = True, validation_data = (X_test, y_test))
end = time.time()
runtime_mnist = end - start
###
print(f'Total runtime: {round(runtime_mnist, 2)}')

In [12]:
###
filepath = "classifiers/%s-{epoch:02d}-{val_accuracy:.4f}-fashion.hdf5"%full_name
checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy', save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

In [15]:
start = time.time()
history = finalModel.fit(X_train, y_train, epochs = 100, verbose = 1,
                         shuffle = True, validation_data = (X_test, y_test))
end = time.time()
runtime_fashion = end - start
###
print(f'Total runtime: {round(runtime_fashion, 2)}')

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 37s 618us/sample - loss: 0.6407 - accuracy: 0.8198 - Precision: 0.8680 - Recall: 0.7729 - val_loss: 0.6488 - val_accuracy: 0.8297 - val_Precision: 0.8770 - val_Recall: 0.7839
Epoch 2/100
60000/60000 [==============================] - 37s 620us/sample - loss: 0.5832 - accuracy: 0.8355 - Precision: 0.8769 - Recall: 0.7971 - val_loss: 0.5661 - val_accuracy: 0.8418 - val_Precision: 0.8915 - val_Recall: 0.8018
Epoch 3/100
60000/60000 [==============================] - 37s 623us/sample - loss: 0.5559 - accuracy: 0.8445 - Precision: 0.8828 - Recall: 0.8070 - val_loss: 0.5739 - val_accuracy: 0.8390 - val_Precision: 0.8748 - val_Recall: 0.8053
Epoch 4/100
60000/60000 [==============================] - 38s 626us/sample - loss: 0.5299 - accuracy: 0.8510 - Precision: 0.8865 - Recall: 0.8166 - val_loss: 0.5245 - val_accuracy: 0.8552 - val_Precision: 0.8972 - val_Recall: 0.8144
Epoch 5/100
60

Epoch 36/100
60000/60000 [==============================] - 38s 628us/sample - loss: 0.3636 - accuracy: 0.9062 - Precision: 0.9242 - Recall: 0.8894 - val_loss: 0.5381 - val_accuracy: 0.8609 - val_Precision: 0.8812 - val_Recall: 0.8438
Epoch 37/100
60000/60000 [==============================] - 38s 627us/sample - loss: 0.3588 - accuracy: 0.9094 - Precision: 0.9271 - Recall: 0.8936 - val_loss: 0.5450 - val_accuracy: 0.8680 - val_Precision: 0.8873 - val_Recall: 0.8512
Epoch 38/100
60000/60000 [==============================] - 38s 631us/sample - loss: 0.3534 - accuracy: 0.9100 - Precision: 0.9273 - Recall: 0.8943 - val_loss: 0.5728 - val_accuracy: 0.8650 - val_Precision: 0.8842 - val_Recall: 0.8503
Epoch 39/100
60000/60000 [==============================] - 38s 628us/sample - loss: 0.3496 - accuracy: 0.9109 - Precision: 0.9278 - Recall: 0.8947 - val_loss: 0.5403 - val_accuracy: 0.8672 - val_Precision: 0.8863 - val_Recall: 0.8541
Epoch 40/100
60000/60000 [==============================] - 

Epoch 71/100
60000/60000 [==============================] - 38s 631us/sample - loss: 0.3018 - accuracy: 0.9310 - Precision: 0.9419 - Recall: 0.9202 - val_loss: 0.5984 - val_accuracy: 0.8681 - val_Precision: 0.8857 - val_Recall: 0.8554
Epoch 72/100
60000/60000 [==============================] - 38s 636us/sample - loss: 0.3070 - accuracy: 0.9302 - Precision: 0.9427 - Recall: 0.9190 - val_loss: 0.6543 - val_accuracy: 0.8655 - val_Precision: 0.8801 - val_Recall: 0.8560
Epoch 73/100
60000/60000 [==============================] - 38s 631us/sample - loss: 0.3122 - accuracy: 0.9314 - Precision: 0.9428 - Recall: 0.9212 - val_loss: 0.6360 - val_accuracy: 0.8664 - val_Precision: 0.8800 - val_Recall: 0.8559
Epoch 74/100
60000/60000 [==============================] - 38s 632us/sample - loss: 0.2947 - accuracy: 0.9330 - Precision: 0.9449 - Recall: 0.9224 - val_loss: 0.6425 - val_accuracy: 0.8657 - val_Precision: 0.8795 - val_Recall: 0.8565
Epoch 75/100
60000/60000 [==============================] - 